# HF Transformers 核心模块学习：Pipelines 进阶

我们已经学习了 Pipeline API 针对各类任务的基本使用。

实际上，在 Transformers 库内部实现中，Pipeline 作为管理：`原始文本-输入Token IDs-模型推理-输出概率-生成结果` 的流水线抽象，背后串联了 Transformers 库的核心模块 `Tokenizer`和 `Models`。

![](docs/images/pipeline_advanced.png)

下面我们开始结合大语言模型（在 Transformers 中也是一种特定任务）学习：

- 使用 Pipeline 如何与现代的大语言模型结合，以完成各类下游任务
- 使用 Tokenizer 编解码文本
- 使用 Models 加载和保存模型

## 使用 Pipeline 调用大语言模型

### Language Modeling

语言建模是一项预测文本序列中的单词的任务。它已经成为非常流行的自然语言处理任务，因为预训练的语言模型可以用于许多其他下游任务的微调。最近，对大型语言模型（LLMs）产生了很大兴趣，这些模型展示了零或少量样本学习能力。这意味着该模型可以解决其未经明确训练过的任务！虽然语言模型可用于生成流畅且令人信服的文本，但需要小心使用，因为文本可能并不总是准确无误。

通过理论篇学习，我们了解到有两种典型的语言模型：

- 自回归：模型目标是预测序列中的下一个 Token（文本），训练时对下文进行了掩码。如：GPT-3。
- 自编码：模型目标是理解上下文后，补全句子中丢失/掩码的 Token（文本）。如：BERT。


### 使用 GPT-2 实现文本生成

![](docs/images/gpt2.png)

模型主页：https://huggingface.co/gpt2

In [1]:
from transformers import pipeline

prompt = "Hugging Face is a community-based open-source platform for machine learning."
generator = pipeline(task="text-generation", model="gpt2")
generator(prompt)

/Users/zk/miniconda3/envs/peft/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/zk/miniconda3/envs/peft/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/Users/zk/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Hugging Face is a community-based open-source platform for machine learning. This has been used by the likes of Quora Group, Hacker News Groups, Ecosyn, S2E, and many more to describe what it could potentially mean'}]

#### 设置文本生成返回条数

In [2]:
prompt = "You are very smart"
generator = pipeline(task="text-generation", model="gpt2", num_return_sequences=3)

In [3]:
generator(prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'You are very smart."\n\nFor me, she is too. I am so sure of that, that this is something that doesn\'t end yet. I don\'t think she has an heir. She\'s a princess, and they can\'t have'},
 {'generated_text': 'You are very smart and very funny."\n\n\'You will soon be considered for president of the United States\' by President Barack Obama\n\nShe added: "This story, I\'ve never heard of before, it\'s just unbelievable and something I'},
 {'generated_text': 'You are very smart and I trust you!"\n\n"Yes, well then—that\'s what I should say!" he quipped.\n\n"Is he not a man of this world?"\n\n"Yes, but no, I\'ve'}]

In [4]:
generator(prompt, num_return_sequences=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'You are very smart – and I think this has helped me," said Thomas. While a recent survey by CBS News found that 48% approve of the way Obama is handling his job, only 10% approve.\n\nObama has been in the White'},
 {'generated_text': 'You are very smart, very smart people, you are not the only ones who can understand why these people are so happy. It is a beautiful world."\n\nBut the other problem, he says, is not being educated, but is being ignored'}]

#### 设置文本生成最大长度

In [5]:
generator(prompt, num_return_sequences=2, max_length=16)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'You are very smart, are your hands so good, why not make a movie'},
 {'generated_text': 'You are very smart and incredibly well supported in the community and I was very nervous'}]

### 使用 BERT-Base-Chinese 实现中文补全

![](docs/images/bert-base-chinese.png)

模型主页：https://huggingface.co/bert-base-chinese

In [5]:
from transformers import pipeline

fill_mask = pipeline(task="fill-mask", model="bert-base-chinese")

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
text = "人民是[MASK]可[MASK][MASK]的"

fill_mask(text, top_k=1)

[[{'score': 0.6703642010688782,
   'token': 679,
   'token_str': '不',
   'sequence': '[CLS] 人 民 是 不 可 [MASK] [MASK] 的 [SEP]'}],
 [{'score': 0.07147283107042313,
   'token': 5543,
   'token_str': '能',
   'sequence': '[CLS] 人 民 是 [MASK] 可 能 [MASK] 的 [SEP]'}],
 [{'score': 0.07442499697208405,
   'token': 5375,
   'token_str': '缺',
   'sequence': '[CLS] 人 民 是 [MASK] 可 [MASK] 缺 的 [SEP]'}]]

#### 设置文本补全的条数

In [7]:
text = "美国的首都是[MASK]"

fill_mask(text, top_k=1)

[{'score': 0.759693443775177,
  'token': 8043,
  'token_str': '？',
  'sequence': '美 国 的 首 都 是 ？'}]

In [8]:
text = "巴黎是[MASK]国的首都。"
fill_mask(text, top_k=1)

[{'score': 0.9911921620368958,
  'token': 3791,
  'token_str': '法',
  'sequence': '巴 黎 是 法 国 的 首 都 。'}]

In [9]:
text = "美国的首都是[MASK]"
fill_mask(text, top_k=3)

[{'score': 0.759693443775177,
  'token': 8043,
  'token_str': '？',
  'sequence': '美 国 的 首 都 是 ？'},
 {'score': 0.21126750111579895,
  'token': 511,
  'token_str': '。',
  'sequence': '美 国 的 首 都 是 。'},
 {'score': 0.02683430165052414,
  'token': 8013,
  'token_str': '！',
  'sequence': '美 国 的 首 都 是 ！'}]

In [13]:
text = "美国的首都是[MASK][MASK]"

fill_mask(text, top_k=1)

[[{'score': 0.09838863462209702,
   'token': 1525,
   'token_str': '哪',
   'sequence': '[CLS] 美 国 的 首 都 是 哪 [MASK] [SEP]'}],
 [{'score': 0.7994449138641357,
   'token': 8043,
   'token_str': '？',
   'sequence': '[CLS] 美 国 的 首 都 是 [MASK] ？ [SEP]'}]]

#### 思考：sequence 中出现的 [CLS] 和 [SEP] 是什么？

## 使用 AutoClass 高效管理 `Tokenizer` 和 `Model`

通常，您想要使用的模型（网络架构）可以从您提供给 `from_pretrained()` 方法的预训练模型的名称或路径中推测出来。

AutoClasses就是为了帮助用户完成这个工作，以便根据`预训练权重/配置文件/词汇表的名称/路径自动检索相关模型`。

比如手动加载`bert-base-chinese`模型以及对应的 `tokenizer` 方法如下：

```python
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModel.from_pretrained("bert-base-chinese")
```

以下是我们实际操作和演示：

### 使用 `from_pretrained` 方法加载指定 Model 和 Tokenizer 

In [14]:
from transformers import AutoTokenizer, AutoModel

model_name = "bert-base-chinese"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

#### 使用 BERT Tokenizer 编码文本

编码 (Encoding) 过程包含两个步骤：

- 分词：使用分词器按某种策略将文本切分为 tokens；
- 映射：将 tokens 转化为对应的 token IDs。

In [18]:
# 第一步：分词
sequence = "中国的首都是北京"
tokens = tokenizer.tokenize(sequence)
print(tokens)

['中', '国', '的', '首', '都', '是', '北', '京']


In [19]:
# 第二步：映射
token_ids = tokenizer.convert_tokens_to_ids(tokens)

In [20]:
print(token_ids)

[704, 1744, 4638, 7674, 6963, 3221, 1266, 776]


#### 使用 Tokenizer.encode 方法端到端处理


In [21]:
token_ids_e2e = tokenizer.encode(sequence)

In [22]:
token_ids_e2e

[101, 704, 1744, 4638, 7674, 6963, 3221, 1266, 776, 102]

#### 思考：为什么前后新增了 101 和 102？

In [23]:
tokenizer.decode(token_ids)

'中 国 的 首 都 是 北 京'

In [24]:
tokenizer.decode(token_ids_e2e)

'[CLS] 中 国 的 首 都 是 北 京 [SEP]'

#### 编解码多段文本

In [25]:
sequence_batch = ["美国的首都是华盛顿特区", "中国的首都是北京"]

In [26]:
token_ids_batch = tokenizer.encode(sequence_batch)

In [27]:
tokenizer.decode(token_ids_batch)

'[CLS] 美 国 的 首 都 是 华 盛 顿 特 区 [SEP] 中 国 的 首 都 是 北 京 [SEP]'

![](docs/images/bert_pretrain.png)

### 实操建议：直接使用 tokenizer.\_\_call\_\_ 方法完成文本编码 + 特殊编码补全

编码后返回结果：

```json
input_ids: token_ids
token_type_ids: token_id 归属的句子编号
attention_mask: 指示哪些token需要被关注（注意力机制）
```

In [30]:
embedding_batch = tokenizer("美国的首都是华盛顿特区", "中国的首都是北京")
print(embedding_batch)

{'input_ids': [101, 5401, 1744, 4638, 7674, 6963, 3221, 1290, 4670, 7561, 4294, 1277, 102, 704, 1744, 4638, 7674, 6963, 3221, 1266, 776, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [31]:
# 优化下输出结构
for key, value in embedding_batch.items():
    print(f"{key}: {value}\n")

input_ids: [101, 5401, 1744, 4638, 7674, 6963, 3221, 1290, 4670, 7561, 4294, 1277, 102, 704, 1744, 4638, 7674, 6963, 3221, 1266, 776, 102]

token_type_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]



### 添加新 Token

当出现了词表或嵌入空间中不存在的新Token，需要使用 Tokenizer 将其添加到词表中。 Transformers 库提供了两种不同方法：

- add_tokens: 添加常规的正文文本 Token，以追加（append）的方式添加到词表末尾。
- add_special_tokens: 添加特殊用途的 Token，优先在已有特殊词表中选择（`bos_token, eos_token, unk_token, sep_token, pad_token, cls_token, mask_token`）。如果预定义均不满足，则都添加到`additional_special_tokens`。

#### 添加常规 Token

先查看已有词表，确保新添加的 Token 不在词表中：

In [32]:
len(tokenizer.vocab.keys())

21128

In [35]:
from itertools import islice

# 使用 islice 查看词表部分内容
for key, value in islice(tokenizer.vocab.items(), 10):
    print(f"{key}: {value}")

##閃: 20329
要: 6206
淮: 3917
sports: 10646
疑: 4542
193: 10185
##倉: 13999
##艰: 18737
碳: 4823
hotel: 8462


In [36]:
new_tokens = ["天干", "地支"]

In [37]:
# 将集合作差结果添加到词表中
new_tokens = set(new_tokens) - set(tokenizer.vocab.keys())

In [38]:
new_tokens

{'地支', '天干'}

In [39]:
tokenizer.add_tokens(list(new_tokens))

2

In [40]:
# 新增加了2个Token，词表总数由 21128 增加到 21130
len(tokenizer.vocab.keys())

21130

#### 添加特殊Token（审慎操作）

In [41]:
new_special_token = {"sep_token": "NEW_SPECIAL_TOKEN"}

In [42]:
tokenizer.add_special_tokens(new_special_token)

1

In [43]:
# 新增加了1个特殊Token，词表总数由 21128 增加到 21131
len(tokenizer.vocab.keys())

21131

### 使用 `save_pretrained` 方法保存指定 Model 和 Tokenizer 

借助 `AutoClass` 的设计理念，保存 Model 和 Tokenizer 的方法也相当高效便捷。

假设我们对`bert-base-chinese`模型以及对应的 `tokenizer` 做了修改，并更名为`new-bert-base-chinese`，方法如下：

```python
tokenizer.save_pretrained("./models/new-bert-base-chinese")
model.save_pretrained("./models/new-bert-base-chinese")
```

保存 Tokenizer 会在指定路径下创建以下文件：
- tokenizer.json: Tokenizer 元数据文件；
- special_tokens_map.json: 特殊字符映射关系配置文件；
- tokenizer_config.json: Tokenizer 基础配置文件，存储构建 Tokenizer 需要的参数；
- vocab.txt: 词表文件；
- added_tokens.json: 单独存放新增 Tokens 的配置文件。

保存 Model 会在指定路径下创建以下文件：
- config.json：模型配置文件，存储模型结构参数，例如 Transformer 层数、特征空间维度等；
- pytorch_model.bin：又称为 state dictionary，存储模型的权重。

In [44]:
tokenizer.save_pretrained("./models/new-bert-base-chinese")

('./models/new-bert-base-chinese/tokenizer_config.json',
 './models/new-bert-base-chinese/special_tokens_map.json',
 './models/new-bert-base-chinese/vocab.txt',
 './models/new-bert-base-chinese/added_tokens.json',
 './models/new-bert-base-chinese/tokenizer.json')

In [46]:
model.save_pretrained("./models/new-bert-base-chinese")